In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import json
import sqlite3
from flask import Flask, jsonify
import plotly.graph_objects as go

In [6]:
def get_db_connection():
    conn = sqlite3.connect('./data/group5.db')
    conn.row_factory = sqlite3.Row
    return conn

In [23]:
hosts_df

,Host_Since_Year,City,Id
0,2008,london,9
1,2008,newyork,11
2,2008,paris,2
3,2009,bangkok,2
4,2009,istanbul,1
...,...,...,...
95,2022,istanbul,3905
96,2022,london,2997
97,2022,newyork,1653
98,2022,paris,1711


In [24]:
conn = get_db_connection()

# query the db for the base table
df = pd.read_sql('select * from main', conn)
df['Host_Since'] = pd.to_datetime(df['Host_Since'])
df['Host_Since_Year'] = df['Host_Since'].dt.year

# group the df into years and count the number of Ids to get signups by year
hosts_grouped = df.groupby(['Host_Since_Year','City']).count()['Id']
hosts_df=hosts_grouped.reset_index()

# plot the figure using plotly
fig = px.bar(hosts_df, x='Host_Since_Year',y='Id',color='City',barmode="stack",
                labels={
                    "Id": "Count of Hosts",
                    "Host_Since_Year": "Year"
                })
# add a centered title
fig.update_layout(title_text='Number of Hosts by Signup Year', title_x=0.5)

In [35]:
conn = get_db_connection()

# query the db for the base table
df = pd.read_sql('select * from main', conn)
df['Host_Since'] = pd.to_datetime(df['Host_Since'])
df['Host_Since_Year'] = df['Host_Since'].dt.year

# group the df into years and count the number of Ids to get signups by year
hosts_grouped = df.groupby(['Host_Since_Year','City']).count()['Id']
hosts_df=hosts_grouped.reset_index()

fig = go.Figure()

for thing in hosts_df['City']:
    fig.add_trace(go.Bar(x=hosts_df['Host_Since_Year'],y=hosts_df[hosts_df['City']==thing]))

updatemenu= []
buttons=[]

for thing in hosts_df['City']:
    buttons.append(dict(method='restyle',
                        label=thing,
                        args=[{'y':[hosts_df[hosts_df['City']==thing]]}])
                    )

updatemenu=[]
your_menu=dict()
updatemenu.append(your_menu)
updatemenu[0]['buttons']=buttons
updatemenu[0]['direction']='down'
updatemenu[0]['showactive']=True

fig.update_layout(updatemenus=updatemenu)
fig.show()

TypeError: Object of type DataFrame is not JSON serializable

In [49]:
def interactive1(a_city = 'bangkok'):
    # create connection to db
    conn = get_db_connection()

    # query the db for the base table
    df = pd.read_sql('select * from main', conn)
    df['Host_Since'] = pd.to_datetime(df['Host_Since'])
    df['Host_Since_Year'] = df['Host_Since'].dt.year
    if a_city == 'all':
        grouped = df.groupby(['Superhost']).agg({'Id':'count'})
        grouped_df = grouped.reset_index()
        
        grouped_df['Superhost'] = grouped_df['Superhost'].map({'t':'Superhost', 'f':'Regular Host'})
                                    
        fig = px.bar(grouped_df, x='Superhost',y='Id', color='Superhost',
                    labels={
                        "Id": "Count of Hosts",
                        "Superhost": "Superhost Status"
                    })
        # add a centered title
        fig.update_layout(title_text='Number of Superhosts (all cities)', title_x=0.5)
    else:
        df = df[df['City'] == a_city]
        grouped = df.groupby(['Superhost']).agg({'Id':'count'})
        grouped_df = grouped.reset_index()
        
        grouped_df['Superhost'] = grouped_df['Superhost'].map({'t':'Superhost', 'f':'Regular Host'})
                                    
        fig = px.bar(grouped_df, x='Superhost',y='Id', color='Superhost',
                    labels={
                        "Id": "Count of Hosts",
                        "Superhost": "Superhost Status"
                    })
        # add a centered title
        fig.update_layout(title_text=f'Number of Superhosts in {a_city}', title_x=0.5)
    return fig

interactive1('paris')

In [50]:
interactive1('all')